# Post BenchmarkDotNet results

- Add a new branch as a workspace
- Append hostname and GitVersion details into the BenchmarkDotNet result JSON.
- Post results overtime in a test branch to storage API
- Fetch results from storage API

## Run BenchmarkDotNet.Storage in Docker

Source code is in this repo [BenchmarkDotNet.Storage](https://github.com/darkato42/BenchmarkDotNet.Storage/tree/main/BenchmarkDotNet.StorageInMemory). You can `git clone` and launch in IDE or start the Docker container version in a separate terminal.

In [ ]:
# Run this in a separate terminal
# Execute in Notebook not recommended
docker run -p 5252:80 darkato/benchmarkdotnet.storageinmemory:1.0

In [ ]:
# Deploy in Azure Container Instance
az container create -g Demo --name benchmarkdotnet-storage --image darkato/benchmarkdotnet.storageinmemory:1.0 --ip-address public --ports 80

## Preparation

In [ ]:
# Run git clone if you want to run BenchmarkDotNet in a separate repo
# Otherwise, create a separate branch
git fetch
git checkout -b test/post-benchmarks

In [ ]:
# Define function to post benchmarks
function Post-Benchmarks {
    # Define URL for posting benchmarks
    $url = "http://localhost:5252/bdnresults"
    
    # Append hostname and gitversion details
    $hostname = hostname
    $gitversion = dotnet gitversion | ConvertFrom-Json

    Get-ChildItem -Path ./BenchmarkDotNet.Artifacts/results -Filter *.json |
    ForEach-Object {
        $json = Get-Content $_.FullName | ConvertFrom-Json
        $json.HostEnvironmentInfo | add-member -Force -Name "Hostname" -value $hostname -MemberType NoteProperty
        $json | add-member -Force -Name "GitVersion" -value $gitversion -MemberType NoteProperty
        $json | ConvertTo-Json -depth 32| set-content $_.FullName
    }

    # Post results to BenchmarkDotNet.Storage API
    Get-ChildItem -Path ./BenchmarkDotNet.Artifacts/results -Filter *.json |
    ForEach-Object {
        $json = Get-Content $_.FullName
        Invoke-WebRequest -Uri $url -Method POST -ContentType 'application/json' -Body $json
    }
}

## Post 1st benchmark report

We start with `public void Benchmark() => Thread.Sleep(A + B + 50);` in `Program.cs`. We will then make a few more commits to decrease sleep time until `Thread.Sleep(A + B)`.

In [ ]:
# Run BenchmarkDotNet
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj

In [ ]:
# View JSON before changes
# cat ./BenchmarkDotNet.Artifacts/results/MyBenchmarks.ThreadSleep-report-brief.json

Post-Benchmarks

In [ ]:
# View JSON after changes
# cat ./BenchmarkDotNet.Artifacts/results/MyBenchmarks.ThreadSleep-report-brief.json

## Update / Rerun / Repost

1. Manually update the `Program.cs` so test runtime gets less.
1. Commit the changes so a new Git hash code is generated.
1. Rerun benchmarks
1. Post benchmarks JSON to our storage API.

In [ ]:
# Thread.Sleep(A + B + 40)
(Get-Content -path ./BenchmarkDotNet.Sample/Program.cs -Raw) `
-replace ([regex]::Escape('Thread.Sleep(A + B + 50)')), 'Thread.Sleep(A + B + 40)'
| Set-Content -path ./BenchmarkDotNet.Sample/Program.cs

git add -A && git commit -m "Thread.Sleep(A + B + 40)"
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj
Post-Benchmarks

In [ ]:
# Thread.Sleep(A + B + 30)
(Get-Content -path ./BenchmarkDotNet.Sample/Program.cs -Raw) `
-replace ([regex]::Escape('Thread.Sleep(A + B + 40)')), 'Thread.Sleep(A + B + 30)'
| Set-Content -path ./BenchmarkDotNet.Sample/Program.cs

git add -A && git commit -m "Thread.Sleep(A + B + 30)"
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj
Post-Benchmarks

In [ ]:
# Thread.Sleep(A + B + 20)
(Get-Content -path ./BenchmarkDotNet.Sample/Program.cs -Raw) `
-replace ([regex]::Escape('Thread.Sleep(A + B + 30)')), 'Thread.Sleep(A + B + 20)'
| Set-Content -path ./BenchmarkDotNet.Sample/Program.cs

git add -A && git commit -m "Thread.Sleep(A + B + 20)"
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj
Post-Benchmarks

In [ ]:
# Thread.Sleep(A + B + 10)
(Get-Content -path ./BenchmarkDotNet.Sample/Program.cs -Raw) `
-replace ([regex]::Escape('Thread.Sleep(A + B + 20)')), 'Thread.Sleep(A + B + 10)'
| Set-Content -path ./BenchmarkDotNet.Sample/Program.cs

git add -A && git commit -m "Thread.Sleep(A + B + 10)"
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj
Post-Benchmarks

In [ ]:
# Thread.Sleep(A + B)
(Get-Content -path ./BenchmarkDotNet.Sample/Program.cs -Raw) `
-replace ([regex]::Escape('Thread.Sleep(A + B + 10)')), 'Thread.Sleep(A + B)'
| Set-Content -path ./BenchmarkDotNet.Sample/Program.cs

git add -A && git commit -m "Thread.Sleep(A + B)"
dotnet run -c Release --project ./BenchmarkDotNet.Sample/BenchmarkDotNet.Sample.csproj
Post-Benchmarks